In [1]:
import os
import pandas as pd
import numpy as np

In [2]:
# Paths to the directories
train_data_dir = '../Data/musicnet/train_data'
train_labels_dir = '../Data/musicnet/train_labels'
test_data_dir = '../Data/musicnet/test_data'
test_labels_dir = '../Data/musicnet/test_labels'

In [3]:
HOP_LENGTH = 512

In [4]:
def calculate_sfft_stats(data_dir, preprocess_audio, frameSize=2048, overlapFac=0.5, bins_per_octave=12, n_octaves=7):
    sum_ = None
    sum_of_squares = None
    count = 0

    # Wrap the file iteration with tqdm for a progress bar
    for file in tqdm(os.listdir(data_dir), desc="Calculating STFT stats"):
        if file.endswith('.wav'):
            sfft = preprocess_audio(os.path.join(data_dir, file))
            if sum_ is None:
                sum_ = np.sum(sfft, axis=0)
                sum_of_squares = np.sum(sfft ** 2, axis=0)
            else:
                sum_ += np.sum(sfft, axis=0)
                sum_of_squares += np.sum(sfft ** 2, axis=0)
            count += sfft.shape[0]

    mean = sum_ / count
    variance = (sum_of_squares - (sum_ ** 2) / count) / count
    std_dev = np.sqrt(variance)

    return mean, std_dev

In [5]:
import numpy as np
from numpy.lib import stride_tricks
import librosa

def sfft(sig, frameSize=4096, overlapFac=0.5, window=np.hanning, mean=None, std=None):
    sig, _ = librosa.load(sig, sr=44100)
    win = window(frameSize)
    hopSize = int(frameSize - np.floor(overlapFac * frameSize))

    samples = np.append(np.zeros(np.int64(np.floor(frameSize/2.0))), sig)
    cols = np.int64(np.ceil((len(samples) - frameSize) / float(hopSize)) + 1)
    samples = np.append(samples, np.zeros(frameSize))

    frames = stride_tricks.as_strided(samples, shape=(cols, frameSize), strides=(samples.strides[0]*hopSize, samples.strides[0])).copy()
    frames *= win
    
    stft = np.fft.rfft(frames)

    # Apply standardization if mean and std are provided
    if mean is not None and std is not None:
        mean = mean.reshape(-1, 1)  # Reshape mean to be broadcastable
        std = std.reshape(-1, 1)    # Reshape std to be broadcastable
        stft = (stft - mean) / std

    return np.real(np.abs(stft))

In [6]:
def logscale_spec(spec, sr=44100, factor=20.):
    timebins, freqbins = np.shape(spec)

    scale = np.linspace(0, 1, freqbins) ** factor
    scale *= (freqbins-1)/max(scale)
    scale = np.unique(np.round(scale)).astype(int)  # Ensure indices are integers

    newspec = np.complex128(np.zeros([timebins, len(scale)]))
    for i in range(len(scale)):
        if i == len(scale)-1:
            newspec[:, i] = np.sum(spec[:, scale[i]:], axis=1)
        else:
            newspec[:, i] = np.sum(spec[:, scale[i]:scale[i+1]], axis=1)

    allfreqs = np.abs(np.fft.fftfreq(freqbins*2, 1./sr)[:freqbins+1])
    freqs = [np.mean(allfreqs[scale[i]:scale[i+1]]) for i in range(len(scale)-1)] + [np.mean(allfreqs[scale[-1]:])]

    return newspec, freqs


In [7]:
def find_note_range(file_paths):
    min_note, max_note = float('inf'), 0
    for file_path in file_paths:
        for file in os.listdir(file_path):
            if file.endswith('.csv'):
                labels_df = pd.read_csv(os.path.join(file_path, file))
                min_note = min(min_note, labels_df['note'].min())
                max_note = max(max_note, labels_df['note'].max())
    return min_note, max_note + 1

all_label_paths = [train_labels_dir, test_labels_dir]
min_note_value, max_note_value = find_note_range(all_label_paths)
note_range = max_note_value - min_note_value

In [8]:
print('Min note value:', min_note_value)
print('Max note value:', max_note_value)
print('Note range:', note_range)

Min note value: 21
Max note value: 105
Note range: 84


In [9]:
def generate_labels(label_file, n_frames, frame_duration, min_note_value=21):
    labels_df = pd.read_csv(label_file)
    labels = np.zeros((n_frames, 88), dtype=int)  # 88 piano keys

    for _, row in labels_df.iterrows():
        start_frame = int(row['start_time'] // frame_duration)
        end_frame = int(row['end_time'] // frame_duration)
        note = int(row['note']) - min_note_value
        if 0 <= note < 88:
            labels[start_frame:end_frame, note] = 1

    return labels

In [10]:
import matplotlib.pyplot as plt
from tqdm import tqdm

In [11]:
def process_files(data_dir, label_dir, preprocess_audio, preprocess_labels, frameSize=4096, overlapFac=0.5):
    X, y = [], []
    frame_duration = frameSize / 44100 - overlapFac * (frameSize / 44100)

    for file in tqdm(os.listdir(data_dir), desc=f"Processing STFT for files in {data_dir}"):
        if file.endswith('.wav'):
            stft_data = sfft(os.path.join(data_dir, file))
            log_spec, freqs = logscale_spec(stft_data)

            n_frames = log_spec.shape[0]
            label_file = os.path.join(label_dir, file.replace('.wav', '.csv'))
            labels = preprocess_labels(label_file, n_frames, frame_duration)

            X.append(log_spec)  # Transpose to align with the CQT shape
            y.append(labels)

    return X, y


In [12]:
mean, std_dev = calculate_sfft_stats(train_data_dir, sfft)
X_train, y_train = process_files(train_data_dir, train_labels_dir, lambda f: sfft(f, mean=mean, std=std_dev), generate_labels)

mean, std_dev = calculate_sfft_stats(test_data_dir, sfft)
X_test, y_test = process_files(test_data_dir, test_labels_dir, lambda f: sfft(f, mean=mean, std=std_dev), generate_labels)

Calculating STFT stats: 100%|██████████| 320/320 [02:43<00:00,  1.95it/s]
Processing STFT for files in ../Data/musicnet/train_data: 100%|██████████| 320/320 [04:25<00:00,  1.20it/s]
Calculating STFT stats: 100%|██████████| 10/10 [00:02<00:00,  4.88it/s]
Processing STFT for files in ../Data/musicnet/test_data: 100%|██████████| 10/10 [00:03<00:00,  2.88it/s]


In [13]:
print('Number of training examples:', len(X_train))
print('Number of training labels:', len(y_train))
print(f"X_train[0] shape: {X_train[0].shape}")
print(f"y_train[0] shape: {y_train[0].shape}")

Number of training examples: 320
Number of training labels: 320
X_train[0] shape: (9627, 387)
y_train[0] shape: (9627, 88)


In [14]:
for i, sample in enumerate(X_train):
    print(f"Sample {i + 1} shape: {sample.shape}")


Sample 1 shape: (9627, 387)
Sample 2 shape: (5408, 387)
Sample 3 shape: (9572, 387)
Sample 4 shape: (7936, 387)
Sample 5 shape: (11758, 387)
Sample 6 shape: (7012, 387)
Sample 7 shape: (15395, 387)
Sample 8 shape: (10561, 387)
Sample 9 shape: (19908, 387)
Sample 10 shape: (14996, 387)
Sample 11 shape: (16886, 387)
Sample 12 shape: (10243, 387)
Sample 13 shape: (7811, 387)
Sample 14 shape: (4949, 387)
Sample 15 shape: (8000, 387)
Sample 16 shape: (15302, 387)
Sample 17 shape: (10087, 387)
Sample 18 shape: (14117, 387)
Sample 19 shape: (12927, 387)
Sample 20 shape: (9438, 387)
Sample 21 shape: (14570, 387)
Sample 22 shape: (9203, 387)
Sample 23 shape: (6264, 387)
Sample 24 shape: (13940, 387)
Sample 25 shape: (8990, 387)
Sample 26 shape: (6619, 387)
Sample 27 shape: (12451, 387)
Sample 28 shape: (15636, 387)
Sample 29 shape: (13556, 387)
Sample 30 shape: (11051, 387)
Sample 31 shape: (9934, 387)
Sample 32 shape: (6974, 387)
Sample 33 shape: (7570, 387)
Sample 34 shape: (10181, 387)
Sampl

In [15]:
def find_max_length(data_list):
    max_length = max(data.shape[0] for data in data_list)
    return max_length

max_length = max(find_max_length(X_train), find_max_length(X_test))


In [16]:
print('Max length:', max_length)

Max length: 23020


In [17]:
def pad_data(data, max_length, size=None):
    padded_data = [np.pad(x, ((0, max_length - x.shape[0]), (0, 0)), 'constant') for x in data[:size]]
    return np.array(padded_data)

def pad_labels(labels, max_length, size=None):
    padded_labels = [np.pad(y, ((0, max_length - y.shape[0]), (0, 0)), 'constant') for y in labels[:size]]
    return np.array(padded_labels)

X_train_padded = pad_data(X_train, max_length, size=100)
y_train_padded = pad_labels(y_train, max_length, size=100)
X_test_padded = pad_data(X_test, max_length)
y_test_padded = pad_labels(y_test, max_length)


In [26]:
print('X_train_padded shape:', X_train_padded.shape)
print('y_train_padded shape:', y_train_padded.shape)
print('X_test_padded shape:', X_test_padded.shape)

X_train_padded shape: (2302000, 387)
y_train_padded shape: (100, 23020, 88)
X_test_padded shape: (10, 23020, 387)


In [19]:
# Slice all spectrograms and make 5x84 images from them
X_train = np.zeros((X_train_padded.shape[0] * X_train_padded.shape[1], 5, 387))
X_train_padded = X_train_padded.reshape(-1, 387)
y_train = y_train_padded.reshape(-1, 88)

for i in range(len(X_train_padded)):
    X_train[i, 2, :] = X_train_padded[i]
    
    if i > 1:
        X_train[i, 0] = X_train_padded[i - 2]
        X_train[i, 1] = X_train_padded[i - 1]
    elif i == 1:
        X_train[i, 1] = X_train_padded[i - 1]
        
    if i < len(X_train_padded) - 2:
        X_train[i, 3] = X_train_padded[i + 1]
        X_train[i, 4] = X_train_padded[i + 2]
    elif i == len(X_train_padded) - 2:
        X_train[i, 3] = X_train_padded[i + 1]


C:\Users\sokos\AppData\Local\Temp\ipykernel_15284\223850307.py:7: ComplexWarning: Casting complex values to real discards the imaginary part
  X_train[i, 2, :] = X_train_padded[i]
C:\Users\sokos\AppData\Local\Temp\ipykernel_15284\223850307.py:16: ComplexWarning: Casting complex values to real discards the imaginary part
  X_train[i, 3] = X_train_padded[i + 1]
C:\Users\sokos\AppData\Local\Temp\ipykernel_15284\223850307.py:17: ComplexWarning: Casting complex values to real discards the imaginary part
  X_train[i, 4] = X_train_padded[i + 2]
C:\Users\sokos\AppData\Local\Temp\ipykernel_15284\223850307.py:13: ComplexWarning: Casting complex values to real discards the imaginary part
  X_train[i, 1] = X_train_padded[i - 1]
C:\Users\sokos\AppData\Local\Temp\ipykernel_15284\223850307.py:10: ComplexWarning: Casting complex values to real discards the imaginary part
  X_train[i, 0] = X_train_padded[i - 2]
C:\Users\sokos\AppData\Local\Temp\ipykernel_15284\223850307.py:11: ComplexWarning: Casting c

In [20]:
print('X_train shape:', X_train.shape)
print('y_train shape:', y_train.shape)

X_train shape: (2302000, 5, 387)
y_train shape: (2302000, 88)


In [21]:
from torch.utils.data import Dataset, DataLoader


class AudioDataset(Dataset):
    def __init__(self):
        self.n_samples = X_train.shape[0]
    
    def __getitem__(self, index):
        return torch.tensor(X_train[index], dtype=torch.float32), torch.tensor(y_train[index], dtype=torch.float32)
    
    def __len__(self):
        return self.n_samples

In [22]:
train_dataset = AudioDataset()
train_loader = DataLoader(dataset=train_dataset, batch_size=512, shuffle=True, num_workers=0)

In [23]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

# Define the PyTorch model
class MusicModel(nn.Module):
    def __init__(self):
        super(MusicModel, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=(3, 3))
        self.pool = nn.MaxPool2d((1, 2))
        self.conv2 = nn.Conv2d(32, 64, kernel_size=(3, 3))
        self.flatten = nn.Flatten()
        self.lstm = nn.LSTM(input_size=64, hidden_size=512, batch_first=True)
        self.fc = nn.Linear(512, 88)

    def forward(self, x):
        # Convolutional layers
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))

        # Reshape for LSTM
        # Assuming x is the output of your conv layers with shape [batch_size, channels, height, width]
        x = x.permute(0, 3, 1, 2)  # Change to [batch_size, width, channels, height]
        x = x.reshape(x.size(0), x.size(1), -1)  # Flatten channels and height into a single dimension

        # LSTM layer
        x, _ = self.lstm(x)

        # Final output layer
        x = self.fc(x)
        return torch.sigmoid(x)

In [24]:
# Check if CUDA is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print('Device:', device)

# Move the model to the device (GPU if available, otherwise CPU)
model = MusicModel().to(device)

Device: cuda


In [25]:
from tqdm import tqdm

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)

# Training loop
num_epochs = 5
for epoch in range(num_epochs):
    running_loss = 0.0
    # Wrap train_loader with tqdm for a progress bar
    train_loader_progress = tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs}")

    for i, (inputs, labels) in enumerate(train_loader_progress):
        # Move input and label data to the same device as the model
        inputs = inputs.reshape(inputs.shape[0], 1, inputs.shape[1], inputs.shape[2]).to(device)
        labels = labels.to(device).long()

        optimizer.zero_grad()

        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

        # Update progress bar description with the running loss
        train_loader_progress.set_description(f"Epoch {epoch+1}/{num_epochs} Loss: {running_loss/(i+1):.3f}")

    # Print loss every epoch
    print(f'Epoch [{epoch + 1}/{num_epochs}] Loss: {running_loss / len(train_loader):.3f}')

print('Finished Training')

Epoch 1/5 Loss: 3.573: 100%|██████████| 4497/4497 [07:02<00:00, 10.65it/s]


Epoch [1/5] Loss: 3.573


Epoch 2/5 Loss: 3.572:  20%|██        | 904/4497 [00:38<02:32, 23.51it/s]


KeyboardInterrupt: 

In [ ]:
import os
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
import librosa

class AudioDataset(Dataset):
    def __init__(self, data_dir, label_dir, chunk_size, bins_per_octave, n_octaves, min_note_value, max_length):
        self.data_dir = data_dir
        self.label_dir = label_dir
        self.chunk_size = chunk_size
        self.bins_per_octave = bins_per_octave
        self.n_octaves = n_octaves
        self.min_note_value = min_note_value
        self.max_length = max_length
        self.files = [f for f in os.listdir(data_dir) if f.endswith('.wav')]
        self.chunk_indices = self._create_chunk_indices()

    def _create_chunk_indices(self):
        chunk_indices = []
        for file in self.files:
            audio_file_path = os.path.join(self.data_dir, file)
            y, _ = librosa.load(audio_file_path, sr=44100)
            total_frames = len(y)
            num_chunks = (total_frames - 1) // (self.chunk_size * HOP_LENGTH) + 1
            for chunk_idx in range(num_chunks):
                chunk_indices.append((file, chunk_idx))
        return chunk_indices

    def __len__(self):
        return len(self.chunk_indices)

    def __getitem__(self, idx):
        file, chunk_idx = self.chunk_indices[idx]
        audio_file_path = os.path.join(self.data_dir, file)
        label_file_path = os.path.join(self.label_dir, file.replace('.wav', '.csv'))

        # Load labels just once for the file
        labels = generate_labels(label_file_path, self.max_length, HOP_LENGTH / 44100, self.min_note_value)

        # Process audio file in chunks
        y, _ = librosa.load(audio_file_path, sr=44100)
        start_sample = chunk_idx * self.chunk_size * HOP_LENGTH
        end_sample = min(start_sample + self.chunk_size * HOP_LENGTH, len(y))
        y_chunk = y[start_sample:end_sample]

        # Generate CQT for the chunk
        C = librosa.cqt(y_chunk, sr=44100, hop_length=HOP_LENGTH, bins_per_octave=self.bins_per_octave, n_bins=self.n_octaves * self.bins_per_octave)
        C_dB = librosa.amplitude_to_db(abs(C)).T

        # Correct padding calculation
        labels_chunk = labels[chunk_idx * self.chunk_size : (chunk_idx + 1) * self.chunk_size]

        # Calculate padding based on max_length
        padding = max(self.max_length - C_dB.shape[0], 0)

        # Apply consistent padding
        C_dB_padded = np.pad(C_dB, ((0, padding), (0, 0)), 'constant')
        labels_padded = np.pad(labels_chunk, ((0, padding), (0, 0)), 'constant')

        # Add channel dimension
        C_dB_padded = np.expand_dims(C_dB_padded, axis=0)

        return torch.tensor(C_dB_padded, dtype=torch.float32), torch.tensor(labels_padded, dtype=torch.float32)


In [ ]:
chunk_size = 16
batch_size = 8
num_time_steps = 5755 // 4

In [ ]:
max_length = 0
for file in os.listdir(train_data_dir):
    if file.endswith('.wav'):
        audio_file_path = os.path.join(train_data_dir, file)
        y, _ = librosa.load(audio_file_path, sr=44100)
        total_frames = len(y)
        num_chunks = (total_frames - 1) // (chunk_size * HOP_LENGTH) + 1
        max_length = max(max_length, num_chunks * chunk_size)
for file in os.listdir(test_data_dir):
    if file.endswith('.wav'):
        audio_file_path = os.path.join(test_data_dir, file)
        y, _ = librosa.load(audio_file_path, sr=44100)
        total_frames = len(y)
        num_chunks = (total_frames - 1) // (chunk_size * HOP_LENGTH) + 1
        max_length = max(max_length, num_chunks * chunk_size)

In [ ]:
print('Max length:', max_length)

In [ ]:

# Create Dataset and DataLoader instances
train_dataset = AudioDataset(train_data_dir, train_labels_dir, chunk_size, 12, 7, min_note_value, max_length)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

In [ ]:
# import torch
# import torch.nn as nn
# import torch.optim as optim
# import torch.nn.functional as F
# 
# # Define the PyTorch model
# class MusicModel(nn.Module):
#     def __init__(self):
#         super(MusicModel, self).__init__()
#         self.conv1 = nn.Conv2d(1, 32, kernel_size=(3, 3), padding='same')
#         self.pool = nn.MaxPool2d((2, 2))
#         self.conv2 = nn.Conv2d(32, 64, kernel_size=(3, 3), padding='same')
#         self.flatten = nn.Flatten()
#         self.lstm = nn.LSTM(input_size=1935360, hidden_size=256, batch_first=True)
#         self.fc = nn.Linear(256, 88)
# 
#     def forward(self, x):
#         # Convolutional layers
#         x = self.pool(F.relu(self.conv1(x)))
#         x = self.pool(F.relu(self.conv2(x)))
# 
#         # Get the dimensions after the final pooling layer
#         batch_size, channels, height, width = x.size()
#         print(f"X shape: {x.shape}")
# 
#         # Flatten
#         x = self.flatten(x)
# 
#         # Calculate number of features for LSTM input
#         num_features = channels * height * width
#         print(f"Number of features: {num_features}")
#         print(f"X shape: {x.shape}")
# 
#         # Reshape for LSTM
#         x = x.view(batch_size, 1, num_features)
#         print(f"X shape: {x.shape}")
# 
#         # LSTM layer
#         x, _ = self.lstm(x)
# 
#         # Final output layer
#         x = self.fc(x)
#         return torch.sigmoid(x)

In [ ]:
# Check if CUDA is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print('Device:', device)

# Move the model to the device (GPU if available, otherwise CPU)
model = MusicModel().to(device)

In [ ]:
# Loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop
num_epochs = 5
for epoch in range(num_epochs):
    running_loss = 0.0
    for i, data in enumerate(train_loader, 0):
        # Move input and label data to the same device as the model
        inputs, labels = data[0].to(device), data[1].to(device)

        optimizer.zero_grad()

        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        if i % 100 == 99:    # print every 100 mini-batches
            print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss / 100:.3f}')
            running_loss = 0.0

print('Finished Training')


In [ ]:
# print('X_train_padded shape:', X_train_padded.shape)
# print('y_train_padded shape:', y_train_padded.shape)

In [ ]:
# # Reduce the size of the training set to speed up training
# X_train_padded = X_train_padded[:100]
# y_train_padded = y_train_padded[:100]

In [ ]:
# X_train_padded = np.expand_dims(X_train_padded, axis=-1)  # Add a channel dimension
# X_test_padded = np.expand_dims(X_test_padded, axis=-1)    # Add a channel dimension


In [ ]:
# print('X_train_padded shape:', X_train_padded.shape)

In [ ]:
def data_generator(data_dir, label_dir, batch_size, chunk_size, bins_per_octave, n_octaves, min_note_value):
    while True:
        for file in os.listdir(data_dir):
            if file.endswith('.wav'):
                audio_file_path = os.path.join(data_dir, file)
                label_file_path = os.path.join(label_dir, file.replace('.wav', '.csv'))

                # Load labels just once for the file
                labels = generate_labels(label_file_path, chunk_size, HOP_LENGTH / 44100, min_note_value)

                # Process audio file in chunks
                y, sr = librosa.load(audio_file_path, sr=44100)
                total_frames = len(y)
                num_chunks = (total_frames - 1) // (chunk_size * HOP_LENGTH) + 1

                for chunk_idx in range(num_chunks):
                    start_sample = chunk_idx * chunk_size * HOP_LENGTH
                    end_sample = min(start_sample + chunk_size * HOP_LENGTH, total_frames)
                    y_chunk = y[start_sample:end_sample]

                    # Generate CQT for the chunk
                    C = librosa.cqt(y_chunk, sr=sr, hop_length=HOP_LENGTH, bins_per_octave=bins_per_octave, n_bins=n_octaves * bins_per_octave)
                    C_dB = librosa.amplitude_to_db(abs(C)).T

                    # Correct padding calculation
                    padding = max(chunk_size - C_dB.shape[0], 0)
                    C_dB_padded = np.pad(C_dB, ((0, padding), (0, 0)), 'constant')

                    # Get corresponding labels
                    labels_chunk = labels[chunk_idx * chunk_size : (chunk_idx + 1) * chunk_size]
                    labels_padded = np.pad(labels_chunk, ((0, padding), (0, 0)), 'constant')

                    # Yield data in batches
                    for i in range(0, len(C_dB_padded), batch_size):
                        X_batch = C_dB_padded[i:i + batch_size]
                        y_batch = labels_padded[i:i + batch_size]
                        yield np.expand_dims(X_batch, axis=-1), y_batch


In [ ]:
batch_size = 8
chunk_size = 16

In [ ]:
# Create generators
train_generator = data_generator(train_data_dir[:50], train_labels_dir[:50], batch_size, chunk_size, 12, 7, min_note_value)
val_generator = data_generator(train_data_dir[50:55], train_labels_dir[50:55], batch_size, chunk_size, 12, 7, min_note_value)

# Calculate steps per epoch for training and validation
steps_per_epoch = 50 // batch_size
validation_steps = 5 // batch_size

# Model training
model = create_music_model(input_shape=(5755, 84, 1))
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

model.fit(
    train_generator,
    steps_per_epoch=steps_per_epoch,
    epochs=5,
    validation_data=val_generator,
    validation_steps=validation_steps,
    verbose=1
)

In [ ]:
# Assuming X_train and y_train are your preprocessed CQT data and labels
model = create_music_model(input_shape=X_train_padded.shape[1:])  # Adjust input shape accordingly
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

model.fit(X_train_padded, y_train_padded, batch_size=8, epochs=1, validation_split=0.2, verbose=1)

In [ ]:
test_loss = model.evaluate(X_test_padded, y_test_padded, verbose=1)
print("Test Loss:", test_loss)